## CONFIGURE PRIMARY CLUSTER

In [ ]:
%env  WORKDIR=/tmp/vault

In [ ]:
%%bash
gcloud auth login
export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_primary")

vault write -f sys/replication/performance/primary/enable primary_cluster_addr=$(terraform output -raw cluster_primary_fqdn_8201)
sleep 5
vault write -format=json sys/replication/performance/primary/secondary-token id=pr12 | jq -r .wrap_info.token > $WORKDIR/pr_token.txt

## CONFIGURE SECONDARY CLUSTER

In [ ]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster_pr_fqdn_8200)
vault_token_pr=$(terraform output -raw cluster_pr_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_pr")

vault write sys/replication/performance/secondary/enable  \
    primary_api_addr=$(terraform output -raw cluster_primary_fqdn_8200) \
    token=$(cat $WORKDIR/pr_token.txt)